## Time-based manual segmentation


In [19]:
import os
import pretty_midi
from midi_player import MIDIPlayer
from midi_player.stylers import dark

midi_file = os.path.join("data", "inputs", "MIDI4.mid")
midi_pm = pretty_midi.PrettyMIDI(midi_file)

MIDIPlayer(midi_file, 400, styler=dark)

In [20]:
from pathlib import Path

def segment_midi(midi_data, segment_length_sec):
  total_length_sec = midi_data.get_end_time()
  num_segments = int(total_length_sec / segment_length_sec)
  segments = []

  for i in range(num_segments):
    start_time = i * segment_length_sec
    end_time = start_time + segment_length_sec
    segment_midi = pretty_midi.PrettyMIDI()

    for instrument in midi_data.instruments:
      segment_instrument = pretty_midi.Instrument(program=instrument.program, name=f"{Path(midi_file).stem}-shifted-{i}")
      for note in instrument.notes:
        if start_time <= note.start < end_time:
          note.start -= start_time
          note.end -= start_time
          segment_instrument.notes.append(note)
      segment_midi.instruments.append(segment_instrument)

    segments.append(segment_midi)

  return segments


In [35]:
def shift_midi_notes(midi_file_path, output_file_path, shift_seconds):
    midi_data = pretty_midi.PrettyMIDI(midi_file_path)

    for instrument in midi_data.instruments:
        for note in instrument.notes:
            note.start += shift_seconds
            note.end += shift_seconds

    midi_data.write(output_file_path)

In [37]:
shifted_file = os.path.join("data", "inputs", "MIDI4-shifted.mid")
shift = -4.5
mds = shift_midi_notes(midi_file, shifted_file, shift)

MIDIPlayer(shifted_file, 300, styler=dark)

In [38]:
import os

bpm = 80
segment_length_bars = 8
padding = 0.0
shifted_midi = pretty_midi.PrettyMIDI(shifted_file)
segments = segment_midi(shifted_midi, 6 + 2 * padding)

dir_out = os.path.join("data", "outputs", "time", f"{Path(shifted_file).stem}")
if not os.path.exists(dir_out):
    os.makedirs(dir_out)
else:
    os.system(f"rm -rf {dir_out}/*")

for i, segment in enumerate(segments):
    segment.write(
    os.path.join(
      "data",
      "outputs",
      "time",
      f"{Path(shifted_file).stem}",
      f"{Path(shifted_file).stem}-{i}.mid",
    )
  )